### Load data + quick look

In [0]:
sdf = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("gs://bucket-autoai/investigations.csv")

df = sdf.toPandas()

In [0]:
import pandas as pd
print("Shape du dataset :", df.shape)
df.head()

In [0]:
display(df)

In [0]:
df.describe()

In [0]:
df.info()

### Structural Cleaning

In [0]:
print(df.dtypes) #df.isnull

In [0]:
# Voir les 20 premières valeurs de ODATE / CDATE
print(df['ODATE'].head(10))
print(df['CDATE'].head(10))

# Voir le type réel
print(df['ODATE'].dtype, df['CDATE'].dtype)

In [0]:
#Convert to datetime
df['ODATE'] = pd.to_datetime(df['ODATE'], format='%Y%m%d', errors='coerce')
df['CDATE'] = pd.to_datetime(df['CDATE'], format='%Y%m%d', errors='coerce')

In [0]:
print(df.duplicated().sum())

In [0]:
print(df.isna().sum()) #df.isnull

In [0]:
## remplir les données vides de type texte par des ''
text_cols = ['NHTSA_ACTION_NUMBER','MAKE','MODEL','COMPNAME','MFR_NAME','CAMPNO','SUBJECT']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].fillna('')

In [0]:
# Colonnes numériques : remplacer NaN par 0 ou moyenne
numeric_cols = ['YEAR']  # ajouter d'autres si besoin
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)

In [0]:
# Colonnes dates : convertir et gérer les NaT
date_cols = ['ODATE','CDATE']
for col in date_cols:
    if col in df.columns:
        # 1️⃣ Convertir en string sans décimales
        df[col] = df[col].astype(str)
        # 2️⃣ Convertir en datetime
        df[col] = pd.to_datetime(df[col], format='%Y%m%d', errors='coerce')

In [0]:
print("\nValeurs manquantes par colonne après traitement :")
print(df.isna().sum())

In [0]:
# df[df['ODATE'].isna()]

In [0]:
import re
df.columns = [re.sub(r'\W+', '_', col.strip()).lower() for col in df.columns]

print("\nNoms de colonnes après uniformisation :")
print(df.columns.tolist())

In [0]:
date_cols = ['ODATE','CDATE']
text_cols = ['NHTSA_ACTION_NUMBER','MAKE','MODEL','COMPNAME','MFR_NAME','CAMPNO','SUBJECT']
# Valider / convertir les types de données

# Texte → string
for col in text_cols:
    col_lower = col.lower()
    if col_lower in df.columns:
        df[col_lower] = df[col_lower].astype(str)

# Numérique → float / int
for col in numeric_cols:
    col_lower = col.lower()
    if col_lower in df.columns:
        df[col_lower] = pd.to_numeric(df[col_lower], errors='coerce')

# Dates → datetime (déjà fait, sécurité)
for col in date_cols:
    col_lower = col.lower()
    if col_lower in df.columns:
        df[col_lower] = pd.to_datetime(df[col_lower], errors='coerce')


In [0]:
print("\nTypes de données après nettoyage :")
print(df.dtypes)

### Text Cleaning